**📚 RAG-System mit LangChain, ChromaDB und Gemini 2**

---

Dieses Notebook implementiert ein einfaches Retrieval-Augmented Generation (RAG) System.
Es verwendet ChromaDB zur Dokumentenspeicherung, LangChain für Workflow-Management und das Modell gemini-2.0-flash.
Das System lädt ein Dokument, unterteilt es in kleinere Chunks, speichert sie in einer Vektor-Datenbank
und nutzt Google Gemini zur Generierung von Antworten auf benutzerdefinierte Fragen unter Berücksichtigung von Kontext und vorherigen Gesprächen.`

---

 ✅ Installieren aller Abhängigkeiten.Zuerst installieren wir alle notwendigen Bibliotheken, die für das RAG-System benötigt werden.Dazu gehören LangChain, LangChain Community, Google Gemini und ChromaDB.
---





In [3]:
!pip uninstall -y google-generativeai
!pip install -q langchain-google-genai==2.1.4 google-ai-generativelanguage==0.6.18


Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5


In [4]:
!pip install -q langchain langchain-community langchain-google-genai langgraph chromadb

✅ Bibliotheken importieren.In diesem Abschnitt importieren wir alle erforderlichen Bibliotheken.Dies umfasst LangChain für die Dokumentenverarbeitung, Google Gemini für das Modell, und ChromaDB für die Speicherung und den Abruf von Vektor-Daten.
---

In [5]:
import os
from getpass import getpass
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.memory.chat_message_histories import FileChatMessageHistory
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.callbacks.tracers import LangChainTracer
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import TypedDict, List
from langchain import hub

✅ Einstellen der Umgebungsvariablen.Hier setzen wir die Umgebungsvariablen für die API-Schlüssel von LangChain und Google Gemini.Diese Schlüssel werden benötigt, um die Funktionen des Systems zu aktivieren.
---

In [7]:
os.environ["LANGCHAIN_API_KEY"] = getpass("🔑LangSmith API Key: ")
os.environ["LANGCHAIN_PROJECT"] = "RAG.2025"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["GOOGLE_API_KEY"] = getpass("🔑API Key (Gemini): ")
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/123.0.0.0 Safari/537.36"

🔑LangSmith API Key: ··········
🔑API Key (Gemini): ··········


✅ Laden des Dokuments (nach August 2024).In diesem Schritt laden wir das Dokument von der angegebenen URL und extrahieren den Text.Die URL verweist auf eine Seite, die Informationen zum rückläufigen Mars in der Astrologie enthält.
---

In [13]:
url = "https://www.astromind.de/astrologie-artikel/r%C3%BCcklaeufiger-mars.html"
loader = WebBaseLoader(url)
documents = loader.load()


✅ Dokument in Chunks aufteilen.Um das Dokument effizient zu verarbeiten und die Antwortqualität zu verbessern, teilen wir es in kleinere Chunks auf.Dies erleichtert das Abrufen relevanter Informationen für spezifische Anfragen.
---

In [14]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(documents)
print(f"🔹 Сhunks: {len(chunks)}")


🔹 Сhunks: 53


✅ Vektorisierung und Speicherung.Hier vektorisieren wir die Chunks des Dokuments mit einem vortrainierten Modell von HuggingFace.Anschließend speichern wir die Vektoren in einer Chroma-Datenbank, die als persistente Speicherung dient.
---

In [28]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
persist_directory = "chroma_db"

if os.path.exists(persist_directory) and os.listdir(persist_directory):
    vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding)   # Wenn die Chroma-Datenbank bereits existiert, laden wir sie.
else:
    vectorstore = Chroma.from_documents(documents=chunks, embedding=embedding, persist_directory=persist_directory) # Ansonsten erstellen wir die Chroma-Datenbank und speichern die Vektoren.
    vectorstore.persist()


✅ Einrichtung der Speicher- und Tracking-Funktionen.In diesem Schritt richten wir die Speicherung des Chat-Verlaufs und die Tracking-Funktion ein,die es uns ermöglicht, die Anfragen und Antworten in unserem RAG-System nachzuverfolgen.
---

In [20]:
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories import FileChatMessageHistory

# Создание и настройка памяти
message_history = FileChatMessageHistory("chat_history.json")

memory = ConversationBufferMemory(
    memory_key="chat_history",
    chat_memory=message_history,
    return_messages=True
)


✅ Einrichtung des Modells und Retrievers.Wir konfigurieren das generative Modell von Google (Gemini) und den Multi-Query Retriever,der mehrere verwandte Dokumente für jede Anfrage abruft, um genauere Antworten zu generieren.
---

In [21]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(),
    llm=llm
)

✅ Abrufen des Prompts aus LangChain Hub.Hier rufen wir das Prompt aus dem LangChain Hub ab, das als Vorlage für das Generieren von Antworten dient.
---

In [22]:
prompt = hub.pull("rlm/rag-prompt")

 ✅ Definition des Zustands und der Funktionen.Wir definieren den Zustand des Systems und zwei Funktionen:- "retrieve" zum Abrufen relevanter Dokumente.- "generate" zur Generierung der Antwort unter Verwendung des Modells und der gespeicherten Kontexte.
 ---

In [30]:

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

def retrieve(state: State):
    docs = multi_query_retriever.get_relevant_documents(state["question"])
    return {"context": docs}

def generate(state: State):
    history = memory.load_memory_variables({})["chat_history"]
    custom_instruction = "Antworte auf Deutsch."
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    full_context = f"{custom_instruction}\n\n{history}\n\nDokumente:\n{docs_content}"
    messages = prompt.invoke({"question": state["question"], "context": full_context})    # Abrufen der Antwort mit dem generativen Modell
    response = llm.invoke(messages, config={"callbacks": [tracer]})
    memory.save_context(inputs={"input": state["question"]}, outputs={"output": response.content})    # Speichern der Antwort im Verlauf
    return {"answer": response.content}

✅ Erstellen des Zustandsgraphen.Wir bauen den Graphen für die Zustandsübergänge, wobei wir die Funktionen "retrieve" und "generate" verknüpfen.
---

In [31]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

✅ Beispiel für die Ausführung.In diesem Beispiel stellen wir eine Frage an das System und generieren die Antwort basierend auf dem Kontext.
---

In [26]:
# ✅ Пример запуска
state = {"question": "Was bedeutet ein rückläufiger Mars?"}
result = graph.invoke(state)
print("📍Antwort:", result["answer"])

📍Antwort: Ein rückläufiger Mars bedeutet, dass der Planet von der Erde aus betrachtet rückwärts durch den Himmel zu wandern scheint. In der Astrologie wird dies als eine Zeit beschrieben, in der die Energie des Planeten nach innen gelenkt wird. Für Mars, der normalerweise für Antrieb, Mut und Tatendrang steht, kann das bedeuten, dass man verlangsamt wird und vor unüberlegten Entscheidungen geschützt wird.


✅Beispiel-Dialog
----

In [27]:
questions = [
    "Warum ist das bedeutsam für den Jahresbeginn?",
    "Was sollte man bei der Planung von Aktivitäten während des rückläufigen Mars beachten?",
    "Welche Empfehlungen gibt die Astrologie, um die negativen Effekte des rückläufigen Mars zu minimieren?",
    "Wie beeinflusst der rückläufige Mars das Privatleben?",
    "Welche Fehler sollte man während des rückläufigen Mars vermeiden?"
]

for q in questions:
    result = graph.invoke({"question": q})
    print(f"❓ Frage: {q}")
    print(f"✅ Antwort: {result['answer']}\n{'-'*80}")

❓ Frage: Warum ist das bedeutsam für den Jahresbeginn?
✅ Antwort: Der rückläufige Mars zu Jahresbeginn 2025 bedeutet, dass man nicht überstürzt mit neuen Ideen ins Jahr starten sollte. Es ist eine Zeit, um Altes loszulassen und Klarheit darüber zu gewinnen, was man wirklich will. Dies schützt vor unüberlegten Entscheidungen und ermöglicht es, sich auf das Wesentliche zu konzentrieren.
--------------------------------------------------------------------------------
❓ Frage: Was sollte man bei der Planung von Aktivitäten während des rückläufigen Mars beachten?
✅ Antwort: Während des rückläufigen Mars sollte man es vermeiden, überstürzt mit neuen Ideen zu starten und sich stattdessen darauf konzentrieren, Altes loszulassen und Klarheit zu gewinnen. Man sollte sich innerlich flexibel halten und offen für Veränderungen sein, falls man in dieser Zeit mit etwas Neuem beginnen muss. Es ist ratsam, innovative Vorhaben aufzuschieben, bis die Phase des rückläufigen Mars vorbei ist.
--------------